What seems to work?
* PB Thresholding on input.
* Loop over buffers - using zip pairs or t+1 ~(zipped pairs maybe better for custom pairings).

What needs adjusting?
* What exactly the outputs and inputs are? Binary signals or probabilities? Subtracted feedback or something else?


In [6]:
from src.var_processor.pb_threshold import pb_threshold
import numpy as np

In [7]:
def check_size(self, frame):
    """Check size of input matches array shape."""
    if frame.shape == self.forward_array.shape[0:2]:
        return True
    else:
        return False

def add_to_array(array, frame):
    """Add a frame to a rolling array."""
    array = np.roll(array, -1, axis=2)
    # Add frame to end of buffer
    array[..., -1] = frame
    return array

def process_array(array):
    """Process an array."""
    # Start with just averaging - could be convolution
    return np.mean(array, axis=2)


class Buffer:
    """Object for a time buffer."""

    def __init__(self, rows, cols, length):
        """Initialise object.

        Assumes 8-bit values (for now).
        """
        # Set up an array to store a rolling window of inputs
        self.forward_array = np.zeros(shape=(rows, cols, length), dtype=np.uint8)
        # Initialise feedback mean
        self.backward_array = np.zeros(shape=(rows, cols, length), dtype=np.uint8)  

    def feedforward(self, frame):
        """Add a frame to the buffer in FF mode."""
        self.forward_array = add_to_array(self.forward_array, frame)
        return None
    
    def feedback(self, frame):
        """Add a frame to the buffer in FB mode."""
        self.backward_array = add_to_array(self.backward_array, frame)
        return None

    @property
    def latest(self):
        """Return latest entry in buffer."""
        return self.forward_array[..., -1]
    
    @property
    def ff_output(self):
        """Provide feedforward output."""
        average = process_array(self.forward_array)
        # Convert to 8-bit for thresholding
        converted = (average*255).astype(np.uint8)
        # Add non-linearity
        binary = pb_threshold(converted)
        return binary
        
    @property
    def fb_output(self):
        """Provide feedback / reconstruction output."""
        average = process_array(self.backward_array)
        # Subtract input - this is just the latest frame
        difference = average - self.latest  # This is float64
        # Clip to 0 to 1
        clipped = np.clip(difference, 0, 1)
        # Convert to 8-bit integer
        converted = (clipped*255).astype(np.uint8)
        # Add non-linearity
        binary = pb_threshold(converted)
        return binary
        
    def iterate(self, input_frame, input_feedback):
        """Perform both a forward and backward pass."""
        self.feedforward(input_frame)
        self.feedback(input_feedback)
        return self.ff_output, self.fb_output


In [8]:
b = Buffer(4, 1, 2)
a = np.arange(0, 4).reshape(4, 1)
b.feedforward(a)
assert np.array_equal(b.latest, a)
b.feedforward(a)
assert np.array_equal(b.latest, a)
b.feedforward(a)
assert np.array_equal(b.latest, a)
average = b.ff_output
assert average.shape == a.shape
assert average.max() <= 1
assert average.min() >= 0
b.feedback(a)
fb_output = b.fb_output
assert fb_output.shape == a.shape
assert fb_output.max() <= 1
assert fb_output.min() >= 0

How do we do the pb_threshold with float64 data representing the difference?

Average has a maximum of 1 and fb has a max of 1 so the range is -1 > 1. Ah so we times by 128 and add 128 to get 8-bit values? But would this affect the suppression of values less than 0?

Do we want instead to clip the negative values to 0 then scale? No - if we just times by 255 this will happen as values less than 0 will be clipped? but then we would have a range of -255 to 255 > 8 bit. And if our positive values stop at 128 then they'd be compared with 255 values...

So we need to clip then times by 255 then threshold as 8-bit values?

In [9]:
self = b
average = process_array(self.forward_array)

In [10]:
average

array([[0.],
       [1.],
       [2.],
       [3.]])

In [11]:
self.forward_array

array([[[0, 0]],

       [[1, 1]],

       [[2, 2]],

       [[3, 3]]], dtype=uint8)

In [12]:
print(average.dtype)

float64


In [13]:
data_type = average.dtype
bit_size = data_type.itemsize*8; print(bit_size)

64


In [14]:
data_type.itemsize

8

In [15]:
self.forward_array.dtype.itemsize

1

In [16]:
# Subtract input - this is just the latest frame
difference = average - self.latest  # This is float64
# Clip to 0 to 1
clipped = np.clip(difference, 0, 1)
# Convert to 8-bit integer
converted = (clipped*255).astype(np.uint8)
# Add non-linearity
binary = pb_threshold(converted)
print(difference, clipped, converted, binary)

[[0.]
 [0.]
 [0.]
 [0.]] [[0.]
 [0.]
 [0.]
 [0.]] [[0]
 [0]
 [0]
 [0]] [[0]
 [0]
 [0]
 [0]]


In [17]:
b = Buffer(4, 1, 2)

In [18]:
a = np.random.randint(255, size=(4,1)); print(a)
b.feedforward(a)
assert np.array_equal(b.latest, a)
b.feedforward(a)
assert np.array_equal(b.latest, a)
b.feedforward(a)
assert np.array_equal(b.latest, a)
average = b.ff_output
assert average.shape == a.shape
assert average.max() <= 1
assert average.min() >= 0
b.feedback(a)
fb_output = b.fb_output
assert fb_output.shape == a.shape
assert fb_output.max() <= 1
assert fb_output.min() >= 0

[[133]
 [ 38]
 [ 22]
 [182]]


In [19]:
print(average, fb_output)

[[1]
 [1]
 [1]
 [0]] [[0]
 [0]
 [0]
 [0]]


# Connecting the Buffers Together

Our time series objects don't need to have any flags or counts with the buffer above.

We look at the averaging as a form of reversable down and up sampling - which just equates to an integrator followed by a threshold.

So we convert binary pulses into a fraction representing the average frequency, which is also an estimate of the intensity in a set range (e.g. 8 bit), then we convert that into a binary pulse.

In [20]:
base_list = range(0,5)
print([i for i in base_list])
for e1, e2 in zip(base_list[:-1], base_list[1:]):
    print(e1, e2)

[0, 1, 2, 3, 4]
0 1
1 2
2 3
3 4


In [33]:
class TimeSeries:
    """Generate a cascade of time buffers."""

    def __init__(self, rows, cols, length, stages):
        """Initialise list for buffer series.
        
        Args:
            rows - integer indicating input array (frame) height in rows.
            cols - integer indicating input array (frame) width in cols.
            length - integer indicating number of time steps to buffer.
            stages - integer indicating the number of buffer stages.
        """
        self.time_series = list()
        self.shape = (rows, cols, length)
        # Generate series of buffers
        self.time_series = [Buffer(*self.shape) for i in range(0, stages)]
        return None

    def add(self, frame):
        """Add frame for processing.
        
        This runs for each time iteration and passes information between buffers.
        """
        # Define variable to hold data passed forward 
        feedforward = frame
        # Iterate through pairs of buffers in series 
        for buffer_ff, buffer_fb in zip(self.time_series[:-1], self.time_series[1:]):
            feedback = buffer_fb.fb_output
            # Get feedforward and feedback for buffer
            feedforward, _ = buffer_ff.iterate(feedforward, feedback)
        return None
    
    @property
    def ff_output(self):
        """Provide feedforward output of all arrays."""
        return np.asarray([buffer.ff_output for buffer in self.time_series]) 
        
    @property
    def fb_output(self):
        """Provide feedback / reconstruction output."""
        return np.asarray([buffer.fb_output for buffer in self.time_series]) 

    @property
    def latest(self):
        """Get last entry of each buffer as array."""
        return np.asarray([buffer.latest for buffer in self.time_series]) 

In [76]:
ts = TimeSeries(4,3,6,8)

So this creates 8 buffers in series that take a 2D input of 4 rows and 3 columns and are adapted to store 6 samples.

In [77]:
print(ts.ff_output, ts.fb_output, ts.latest)

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]] [[[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]] [[[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0

In [78]:
ts.ff_output.shape

(8, 4, 3)

In [79]:
def test_timeseries():
    """Test the time series object."""
    ts = TimeSeries(4,3,6,8)
    # Check everything is initialised to 0
    assert np.array_equal(ts.ff_output, np.zeros((8, 4,3)))
    assert np.array_equal(ts.fb_output, np.zeros((8, 4,3)))
    assert np.array_equal(ts.latest, np.zeros((8, 4,3)))
    a = np.random.randint(255, size=(4,3), dtype=np.uint8)
    pb_a = pb_threshold(a)
    ts.add(pb_a)

In [98]:
a = np.arange(0, 4, dtype=np.uint8); print(a)
b = a.reshape(1,1,-1); print(b.shape)
process_array(b)

[0 1 2 3]
(1, 1, 4)


array([[1.5]])

In [99]:
def test_processarray():
    """Test process array function."""
    a = np.arange(0, 4, dtype=np.uint8); 
    b = a.reshape(1,1,-1); 
    assert process_array(b) == 1.5

In [100]:
test_processarray()

In [80]:
test_timeseries()

In [81]:
a = np.random.randint(255, size=(4,3), dtype=np.uint8)
pb_a = pb_threshold(a)
ts.add(pb_a)

In [82]:
print(ts.ff_output, ts.fb_output, ts.latest)

[[[0 0 0]
  [0 0 0]
  [0 1 0]
  [0 0 1]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]] [[[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]] [[[1 0 0]
  [0 1 0]
  [0 1 1]
  [0 1 1]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0

In [84]:
for i in range(0, 100):
    a = np.random.randint(255, size=(4,3), dtype=np.uint8)
    pb_a = pb_threshold(a)
    ts.add(pb_a)

In [85]:
print(ts.ff_output, ts.fb_output, ts.latest, sep="\n\n")

[[[0 1 1]
  [1 0 1]
  [0 0 1]
  [1 1 0]]

 [[0 1 0]
  [0 0 0]
  [0 0 1]
  [1 1 0]]

 [[1 1 0]
  [1 1 1]
  [1 1 1]
  [1 0 1]]

 [[0 1 0]
  [0 1 1]
  [0 1 1]
  [1 0 0]]

 [[0 0 1]
  [1 1 1]
  [1 0 0]
  [0 0 0]]

 [[1 0 1]
  [1 1 1]
  [1 0 0]
  [0 0 0]]

 [[1 0 1]
  [1 1 0]
  [1 1 0]
  [1 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]

[[[0 0 0]
  [0 1 1]
  [1 0 0]
  [1 0 1]]

 [[0 0 0]
  [0 0 1]
  [0 1 1]
  [1 0 0]]

 [[1 1 1]
  [1 0 1]
  [0 0 1]
  [1 0 1]]

 [[0 1 1]
  [0 0 1]
  [0 0 1]
  [1 0 0]]

 [[0 1 1]
  [1 1 1]
  [0 1 0]
  [0 0 0]]

 [[0 1 1]
  [0 1 1]
  [1 0 0]
  [1 0 0]]

 [[1 1 1]
  [0 1 1]
  [0 0 0]
  [1 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  

Feedback output is always 0. Is this because the data is random and there are no patterns? Or because a bug in the clipping code?

In [89]:
print(ts.time_series[0].forward_array.shape, ts.time_series[0].forward_array, sep="\n\n")

(4, 3, 6)

[[[0 1 0 1 0 0]
  [0 0 1 1 0 0]
  [1 1 0 0 1 0]]

 [[1 0 1 0 0 0]
  [1 0 1 0 0 1]
  [0 1 1 1 0 1]]

 [[0 0 1 1 0 1]
  [1 0 0 1 0 0]
  [1 1 1 1 0 0]]

 [[1 1 0 0 1 1]
  [1 1 0 0 1 0]
  [1 1 0 1 0 1]]]


In [86]:
process_array(ts.time_series[0].forward_array)

array([[0.33333333, 0.33333333, 0.5       ],
       [0.33333333, 0.5       , 0.66666667],
       [0.5       , 0.33333333, 0.66666667],
       [0.66666667, 0.5       , 0.66666667]])

In [90]:
for i in range(0,6):
    print(ts.time_series[0].forward_array[:,:, i])

[[0 0 1]
 [1 1 0]
 [0 1 1]
 [1 1 1]]
[[1 0 1]
 [0 0 1]
 [0 0 1]
 [1 1 1]]
[[0 1 0]
 [1 1 1]
 [1 0 1]
 [0 0 0]]
[[1 1 0]
 [0 0 1]
 [1 1 1]
 [0 0 1]]
[[0 0 1]
 [0 0 0]
 [0 0 0]
 [1 1 0]]
[[0 0 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]]
